# Log Simplifier for stakeholders
## Production-Ready Log Analysis with Ollama Integration



This notebook demonstrates how to transform complex technical logs into executive-friendly summaries using local LLM (Ollama) with enterprise-grade features.

### Perfect for DevOps/SRE professionals who need to communicate technical issues to business stakeholders!

## Prerequisites and Setup

Before running this notebook, ensure you have:
1. **Ollama installed and running** on localhost:11434
2. **Required Python packages** installed
3. **Model downloaded** (e.g., llama3)

### Installation Commands:
```
# Install Ollama
curl -fsSL https://ollama.com/install.sh | sh

# Start Ollama service
ollama serve

# Download a model
ollama pull llama3

# Install Python dependencies
pip install requests pyyaml
```

In [30]:
# Import required libraries
import os
import json
import requests
import logging
import yaml
from datetime import datetime
from typing import Dict, List, Optional, Tuple
from pathlib import Path
import re
import time
from collections import defaultdict, Counter

print("🚀 Improved Stakeholder Log Summarizer - Ready!")
print("📋 Enterprise-grade log analysis with Ollama integration")
print("💼 Perfect for generating executive summaries from technical logs")

🚀 Improved Stakeholder Log Summarizer - Ready!
📋 Enterprise-grade log analysis with Ollama integration
💼 Perfect for generating executive summaries from technical logs


## Step 1: Check Ollama Connection

In [31]:
def check_ollama_connection():
    """Check if Ollama is running and accessible."""
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=5)
        if response.status_code == 200:
            models = response.json().get('models', [])
            print("✅ Ollama is running!")
            print(f"📋 Available models: {[m['name'] for m in models]}")
            return True
        else:
            print(f"❌ Ollama responded with status {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Ollama. Make sure it's running on localhost:11434")
        print("💡 Start Ollama with: ollama serve")
        return False
    except Exception as e:
        print(f"❌ Error checking Ollama: {e}")
        return False

# Check Ollama connection
ollama_available = check_ollama_connection()

✅ Ollama is running!
📋 Available models: ['llama3:latest']


## Step 2: Configuration Management Class

In [32]:
class LogSummarizerConfig:
    """Configuration management for the log summarizer."""
    
    def __init__(self, config_dict=None):
        """Initialize configuration from dictionary or defaults."""
        self.config = config_dict or self._get_default_config()
        
    def _get_default_config(self) -> Dict:
        """Get default configuration."""
        return {
            'ollama': {
                'url': 'http://localhost:11434/api/generate',
                'model': 'llama3',
                'timeout': 120,
                'max_retries': 3,
                'retry_delay': 2
            },
            'logging': {
                'level': 'INFO',
                'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            },
            'output': {
                'formats': ['markdown', 'json'],
                'base_filename': 'stakeholder_summary',
                'include_timestamp': True
            },
            'analysis': {
                'max_log_size': 50000,
                'summary_types': ['executive', 'technical', 'action_items'],
                'business_focus': True
            },
            'security': {
                'sanitize_data': True,
                'redact_patterns': [
                    r"password=\S+",
                    r"api_key=\S+",
                    r"token=\S+",
                    r"\b\d{4}[- ]?\d{4}[- ]?\d{4}[- ]?\d{4}\b"
                ]
            }
        }
    
    def get(self, key_path: str, default=None):
        """Get configuration value using dot notation."""
        keys = key_path.split('.')
        value = self.config
        
        for key in keys:
            if isinstance(value, dict) and key in value:
                value = value[key]
            else:
                return default
        
        return value

# Initialize configuration
config = LogSummarizerConfig()
print("✅ Configuration initialized with production defaults")
print(f"🔧 Ollama model: {config.get('ollama.model')}")
print(f"📊 Max log size: {config.get('analysis.max_log_size')} characters")

✅ Configuration initialized with production defaults
🔧 Ollama model: llama3
📊 Max log size: 50000 characters


## Step 3: Main Stakeholder Log Summarizer Class

In [33]:
# Main Stakeholder Log Summarizer Class (full implementation)
class StakeholderLogSummarizer:
    def __init__(self, config_obj=None):
        self.config = config_obj or LogSummarizerConfig()
        logging.basicConfig(
            level=getattr(logging, self.config.get('logging.level', 'INFO')),
            format=self.config.get('logging.format', '%(asctime)s - %(levelname)s - %(message)s')
        )
        self.logger = logging.getLogger(__name__)
        self.ollama_url = self.config.get('ollama.url')
        self.model = self.config.get('ollama.model')
        self.timeout = self.config.get('ollama.timeout', 120)
        self.max_retries = self.config.get('ollama.max_retries', 3)
        self.retry_delay = self.config.get('ollama.retry_delay', 2)

    def sanitize_log_content(self, content: str) -> str:
        if not self.config.get('security.sanitize_data', True):
            return content
        sanitized = content
        redact_patterns = self.config.get('security.redact_patterns', [])
        for pattern in redact_patterns:
            sanitized = re.sub(pattern, '[REDACTED]', sanitized, flags=re.IGNORECASE)
        return sanitized

    def analyze_log_content(self, log_content: str) -> Dict:
        try:
            analysis = {
                'total_lines': len(log_content.split('\n')),
                'error_count': 0,
                'warning_count': 0,
                'critical_count': 0,
                'fatal_count': 0,
                'business_indicators': []
            }
            lines = log_content.split('\n')
            for line in lines:
                line_upper = line.upper()
                if 'ERROR' in line_upper:
                    analysis['error_count'] += 1
                elif 'WARNING' in line_upper or 'WARN' in line_upper:
                    analysis['warning_count'] += 1
                elif 'CRITICAL' in line_upper:
                    analysis['critical_count'] += 1
                elif 'FATAL' in line_upper:
                    analysis['fatal_count'] += 1
            business_patterns = [
                'payment', 'transaction', 'order', 'customer', 'revenue',
                'database', 'connection', 'timeout', 'memory', 'security',
                'authentication', 'authorization', 'breach', 'attack'
            ]
            for pattern in business_patterns:
                if pattern.lower() in log_content.lower():
                    analysis['business_indicators'].append(pattern)
            timestamp_pattern = r'\d{4}-\d{2}-\d{2}[T ]\d{2}:\d{2}:\d{2}'
            timestamps = re.findall(timestamp_pattern, log_content)
            if timestamps:
                analysis['time_span'] = f"{timestamps[0]} to {timestamps[-1]}"
            return analysis
        except Exception as e:
            self.logger.error(f"Error analyzing log content: {str(e)}")
            return {'error': str(e)}

    def generate_enhanced_prompt(self, log_content: str, summary_type: str, analysis: Dict) -> str:
        base_context = f"""
You are an expert technical communicator specializing in translating complex system logs 
into clear, actionable insights for business stakeholders.

LOG ANALYSIS SUMMARY:
- Total log entries: {analysis.get('total_lines', 'Unknown')}
- Critical issues: {analysis.get('critical_count', 0)} fatal, {analysis.get('error_count', 0)} errors
- Warning indicators: {analysis.get('warning_count', 0)}
- Time span: {analysis.get('time_span', 'Unknown')}
- Business systems involved: {', '.join(analysis.get('business_indicators', []))}
"""
        if summary_type == 'executive':
            prompt = base_context + """
Create an EXECUTIVE SUMMARY focused on:
1. BUSINESS IMPACT: What does this mean for revenue, customers, and operations?
2. URGENCY LEVEL: How critical is the situation? 
3. RESOURCE IMPLICATIONS: What teams/budget are needed?
4. CUSTOMER EXPERIENCE: How are users affected?
5. RISK ASSESSMENT: What are the potential consequences if not addressed?
6. COMMUNICATION NEEDS: Should customers/partners be notified?

Use business language, avoid technical jargon, focus on decision-making needs.
Format with clear sections and bullet points for executive consumption.
"""
        elif summary_type == 'action_items':
            prompt = base_context + """
Create a PRIORITIZED ACTION PLAN with:
1. IMMEDIATE ACTIONS (0-30 minutes): Critical fixes needed right now
2. SHORT-TERM ACTIONS (1-4 hours): Important follow-up tasks  
3. MEDIUM-TERM ACTIONS (1-7 days): Preventive measures and improvements
4. For each action specify: WHO (team/role), WHAT (specific task), WHEN (timeline)
5. RESOURCE REQUIREMENTS: Budget, tools, or external help needed
6. SUCCESS METRICS: How to measure if actions are effective

Format as a clear, prioritized task list with ownership and timelines.
"""
        elif summary_type == 'technical':
            prompt = base_context + """
Create a TECHNICAL SUMMARY for engineering stakeholders including:
1. ROOT CAUSE ANALYSIS: What systems/components failed and why?
2. SYSTEM IMPACT: Which services, databases, or infrastructure are affected?
3. TECHNICAL PATTERNS: Common error types and failure modes
4. PERFORMANCE INDICATORS: Response times, resource usage, capacity issues  
5. DEPENDENCIES: What downstream systems might be impacted?
6. TECHNICAL RECOMMENDATIONS: Specific engineering solutions needed

Use appropriate technical detail while remaining clear and actionable.
"""
        else:
            prompt = base_context + """
Create a COMPREHENSIVE SUMMARY that includes:
1. What happened during this time period?
2. Which business functions were impacted?
3. What are the most critical issues requiring attention?
4. What immediate actions should be taken?
5. What is the estimated business impact?

Balance technical accuracy with business relevance.
"""
        prompt += f"""

ORIGINAL LOGS TO ANALYZE:
{'-' * 50}
{log_content}
{'-' * 50}

Generate your {summary_type.upper()} summary now:
"""
        return prompt

    def call_ollama_with_retry(self, prompt: str) -> str:
        payload = {
            "model": self.model,
            "prompt": prompt,
            "stream": False,
            "options": {
                "temperature": 0.3,
                "top_p": 0.9
            }
        }
        for attempt in range(self.max_retries):
            try:
                self.logger.info(f"Calling Ollama API (attempt {attempt + 1}/{self.max_retries})")
                # Remove timeout argument to allow unlimited wait
                response = requests.post(
                    self.ollama_url,
                    json=payload,
                    headers={'Content-Type': 'application/json'}
                )
                if response.status_code == 200:
                    result = response.json()
                    summary = result.get("response", "")
                    if summary.strip():
                        self.logger.info(f"Successfully generated summary ({len(summary)} characters)")
                        return summary
                    else:
                        raise ValueError("Empty response from Ollama")
                else:
                    raise requests.RequestException(f"HTTP {response.status_code}: {response.text}")
            except requests.exceptions.ConnectionError:
                self.logger.error("Cannot connect to Ollama. Is it running on localhost:11434?")
                if attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay * 2)
                else:
                    raise ConnectionError("Could not connect to Ollama after multiple attempts")
            except Exception as e:
                self.logger.error(f"Error calling Ollama on attempt {attempt + 1}: {str(e)}")
                if attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay)
                else:
                    raise
        raise Exception(f"Failed to get response from Ollama after {self.max_retries} attempts")

    def generate_summaries(self, log_content: str, summary_types: Optional[List[str]] = None) -> Dict:
        if summary_types is None:
            summary_types = self.config.get('analysis.summary_types', ['executive'])
        sanitized_content = self.sanitize_log_content(log_content)
        max_size = self.config.get('analysis.max_log_size', 50000)
        if len(sanitized_content) > max_size:
            sanitized_content = sanitized_content[-max_size:]
        analysis = self.analyze_log_content(sanitized_content)
        summaries = {
            'metadata': {
                'timestamp': datetime.now().isoformat(),
                'model_used': self.model,
                'analysis': analysis
            },
            'summaries': {}
        }
        for summary_type in summary_types:
            try:
                self.logger.info(f"Generating {summary_type} summary...")
                prompt = self.generate_enhanced_prompt(sanitized_content, summary_type, analysis)
                summary = self.call_ollama_with_retry(prompt)
                summaries['summaries'][summary_type] = summary
            except Exception as e:
                self.logger.error(f"Failed to generate {summary_type} summary: {str(e)}")
                summaries['summaries'][summary_type] = f"Error generating summary: {str(e)}"
        return summaries

    def process_log_content(self, log_content: str, summary_types: Optional[List[str]] = None) -> Dict:
        try:
            self.logger.info("Processing log content...")
            summaries = self.generate_summaries(log_content, summary_types)
            self.logger.info("Summary generation complete")
            return {
                'status': 'success',
                'summaries': summaries
            }
        except Exception as e:
            self.logger.error(f"Error processing log content: {str(e)}")
            return {
                'status': 'error',
                'error': str(e)
            }

# Initialize the summarizer
if ollama_available:
    summarizer = StakeholderLogSummarizer(config)
    print("✅ Stakeholder Log Summarizer initialized successfully!")
    print("🔧 Configuration loaded with production-ready settings")
    print("📊 Ready to process complex technical logs")
else:
    print("⚠️ Ollama not available - some features will be limited")

✅ Stakeholder Log Summarizer initialized successfully!
🔧 Configuration loaded with production-ready settings
📊 Ready to process complex technical logs


## Step 4: Create Complex Sample Log Data

Let's create realistic, complex log data that stakeholders would find difficult to understand.

In [34]:
# Create realistic complex log data that stakeholders struggle with
complex_application_log = """
2024-08-27 14:23:01.234 [main] ERROR com.company.payment.PaymentProcessor - Failed to process payment transaction
java.sql.SQLException: Connection is not available, request timed out after 30000ms.
    at com.zaxxer.hikari.pool.HikariPool.createTimeoutException(HikariPool.java:695)
    at com.zaxxer.hikari.pool.HikariPool.getConnection(HikariPool.java:181)
    at com.company.payment.dao.PaymentDAO.insertPayment(PaymentDAO.java:87)
    at com.company.payment.PaymentProcessor.processPayment(PaymentProcessor.java:145)
Caused by: org.postgresql.util.PSQLException: FATAL: remaining connection slots are reserved for non-replication superuser connections

2024-08-27 14:23:02.123 [scheduler-thread-1] ERROR com.company.jobs.DataSyncJob - OutOfMemoryError during data synchronization
java.lang.OutOfMemoryError: Java heap space
    at java.util.Arrays.copyOf(Arrays.java:3332)
    at java.lang.AbstractStringBuilder.ensureCapacityInternal(AbstractStringBuilder.java:124)
    at com.company.jobs.DataSyncJob.buildLargeDataSet(DataSyncJob.java:234)

2024-08-27 14:23:05.456 [main] FATAL com.company.security.AuthenticationManager - Critical security breach detected
com.company.security.SecurityException: Multiple failed authentication attempts from IP 192.168.1.100
    User attempts: admin (15 failures), root (12 failures), administrator (8 failures)
    Attack vector: Password brute force via /api/v1/auth/login
    Risk level: HIGH - Immediate action required

2024-08-27 14:23:07.234 [metrics-collector] ERROR com.company.monitoring.MetricsCollector - Failed to publish metrics to CloudWatch
software.amazon.awssdk.services.cloudwatch.model.CloudWatchException: User is not authorized to perform: cloudwatch:PutMetricData

2024-08-27 14:23:09.567 [worker-thread-3] ERROR com.company.integration.ThirdPartyAPI - HTTP 429 Rate limit exceeded
okhttp3.ResponseException: HTTP 429: {"error":"rate_limit_exceeded","message":"API rate limit of 1000 requests per hour exceeded","retry_after":3600}

2024-08-27 14:23:12.890 [main] ERROR com.company.startup.ApplicationBootstrap - Application failed to start
org.springframework.beans.factory.BeanCreationException: Could not resolve placeholder 'database.password' in value "${database.password}"

2024-08-27 14:23:18.456 [http-nio-8080-exec-23] ERROR com.company.api.FileUploadController - File processing failed
java.io.IOException: No space left on device
Disk usage: /tmp 98% (23.1GB used of 23.5GB available)
File details: upload_20240827_142318.pdf (Size: 145MB, User: customer_78945)
"""

print("📄 Created complex application log with:")
print("   • Java stack traces and OutOfMemoryErrors")
print("   • Database connection pool exhaustion")
print("   • Security brute force attack detection")
print("   • AWS permissions and API rate limiting issues")
print("   • Configuration and disk space problems")
print("")
print("❓ What stakeholders typically ask:")
print("   • What does this mean for our business?")
print("   • Are customers affected?")
print("   • How much money are we losing?")
print("   • What needs to be done immediately?")
print("   • When will it be fixed?")

📄 Created complex application log with:
   • Java stack traces and OutOfMemoryErrors
   • Database connection pool exhaustion
   • Security brute force attack detection
   • AWS permissions and API rate limiting issues
   • Configuration and disk space problems

❓ What stakeholders typically ask:
   • What does this mean for our business?
   • Are customers affected?
   • How much money are we losing?
   • What needs to be done immediately?
   • When will it be fixed?


## Step 5: Analyze the Complex Logs

In [35]:
if ollama_available:
    # Analyze the complex log content first
    analysis = summarizer.analyze_log_content(complex_application_log)
    print("📊 LOG ANALYSIS RESULTS:")
    print("=" * 40)
    print(f"Total entries: {analysis.get('total_lines', 0)}")
    print(f"Critical/Fatal: {analysis.get('critical_count', 0) + analysis.get('fatal_count', 0)}")
    print(f"Errors: {analysis.get('error_count', 0)}")
    print(f"Warnings: {analysis.get('warning_count', 0)}")
    print(f"Time span: {analysis.get('time_span', 'Unknown')}")
    print(f"Business indicators: {analysis.get('business_indicators', [])}")
    total_critical_issues = analysis.get('critical_count', 0) + analysis.get('error_count', 0) + analysis.get('fatal_count', 0)
    print(f"\n⚠️  BUSINESS IMPACT ASSESSMENT:")
    if total_critical_issues >= 5:
        print(f"   🚨 HIGH IMPACT: {total_critical_issues} critical issues detected")
        print("   💰 Estimated revenue impact: $15,000+ per hour")
        print("   📢 Executive notification recommended")
    elif total_critical_issues >= 2:
        print(f"   ⚠️  MEDIUM IMPACT: {total_critical_issues} issues require attention")
        print("   💰 Estimated revenue impact: $5,000+ per hour")
    else:
        print(f"   ✅ LOW IMPACT: {total_critical_issues} minor issues")
else:
    print("⚠️ Ollama not available - using basic analysis")
    print("📄 Complex log contains Java errors, security issues, and system failures")

📊 LOG ANALYSIS RESULTS:
Total entries: 35
Critical/Fatal: 2
Errors: 8
Warnings: 0
Time span: 2024-08-27 14:23:01 to 2024-08-27 14:23:18
Business indicators: ['payment', 'transaction', 'customer', 'database', 'connection', 'timeout', 'memory', 'security', 'authentication', 'breach', 'attack']

⚠️  BUSINESS IMPACT ASSESSMENT:
   🚨 HIGH IMPACT: 10 critical issues detected
   💰 Estimated revenue impact: $15,000+ per hour
   📢 Executive notification recommended


## Step 6: Generate Executive Summary

Transform the complex technical logs into an executive-friendly summary.

In [36]:
if ollama_available:
    print("⏳ Generating executive summary...")
    print("   (This may take 30-60 seconds depending on your system)")
    try:
        result = summarizer.process_log_content(
            complex_application_log,
            summary_types=['executive']
        )
        if result['status'] == 'success':
            print("\n" + "=" * 80)
            print("🎯 EXECUTIVE SUMMARY - TRANSFORMED FOR STAKEHOLDERS")
            print("=" * 80)
            executive_summary = result['summaries']['summaries'].get('executive', '')
            print(executive_summary)
            print("\n" + "=" * 80)
            print("✨ TRANSFORMATION COMPLETE!")
            print("   • Technical jargon → Business language")
            print("   • Stack traces → Revenue impact")
            print("   • Error codes → Action items")
            print("   • 30+ min analysis → 2 min understanding")
        else:
            print(f"❌ Error generating summary: {result['error']}")
    except Exception as e:
        print(f"❌ Unexpected error: {str(e)}")
        print("💡 This might be due to Ollama model not being available or network issues")
else:
    print("⚠️ Ollama not available")
    print("\n🎯 SAMPLE EXECUTIVE SUMMARY (What would be generated):")
    print("=" * 60)
    sample_summary = """
🚨 CRITICAL: Payment System Failure - Immediate Action Required

BUSINESS IMPACT:
• Payment processing system experienced multiple critical failures
• Database connection issues preventing customer transactions
• Security breach detected with ongoing brute force attack
• Estimated revenue impact: $45,000 per hour during outage

IMMEDIATE CONCERNS:
1. PAYMENT PROCESSING DOWN: Customers cannot complete purchases
2. SECURITY INCIDENT: Active cyber attack in progress
3. SYSTEM OVERLOAD: Memory exhaustion causing crashes
4. STORAGE CRISIS: Disk space at 98% capacity

REQUIRED ACTIONS:
1. Increase database connection pool (DevOps - 15 minutes)
2. Block attacking IP address (Security - 5 minutes)
3. Restart services with more memory (SRE - 10 minutes)
4. Clean up disk space immediately (Infrastructure - 10 minutes)

ESTIMATED RESOLUTION: 30-45 minutes
COST OF DELAY: $15,000 per additional hour

EXECUTIVE DECISION REQUIRED:
Should we proactively communicate with customers about service issues?
"""
    print(sample_summary)

2025-08-27 20:23:22,337 - __main__ - INFO - Processing log content...
2025-08-27 20:23:22,339 - __main__ - INFO - Generating executive summary...
2025-08-27 20:23:22,339 - __main__ - INFO - Calling Ollama API (attempt 1/3)
2025-08-27 20:23:22,339 - __main__ - INFO - Generating executive summary...
2025-08-27 20:23:22,339 - __main__ - INFO - Calling Ollama API (attempt 1/3)


⏳ Generating executive summary...
   (This may take 30-60 seconds depending on your system)


2025-08-27 20:25:12,740 - __main__ - INFO - Successfully generated summary (1910 characters)
2025-08-27 20:25:12,745 - __main__ - INFO - Summary generation complete
2025-08-27 20:25:12,745 - __main__ - INFO - Summary generation complete



🎯 EXECUTIVE SUMMARY - TRANSFORMED FOR STAKEHOLDERS
**EXECUTIVE SUMMARY**

**Business Impact:**
The system log analysis indicates a critical situation that may impact revenue, customer satisfaction, and operational efficiency. The failure to process payment transactions, data synchronization issues, security breaches, and application failures may result in lost sales, delayed services, or compromised customer data.

**Urgency Level:**
High urgency is required to address the critical security breach, failed authentication attempts, and potential data loss. Immediate action is necessary to prevent further attacks and ensure business continuity.

**Resource Implications:**
To resolve these issues, we recommend allocating resources from:

* IT Operations team for infrastructure and connectivity issues
* Security team for investigating and mitigating the security breach
* Development team for fixing application failures and implementing rate limiting measures
* Budget allocation for upgradi

## Step 7: Generate Multiple Summary Types

Create Executive, Technical, and Action Items summaries for different stakeholder audiences.

In [37]:
if ollama_available:
    print("⏳ Generating all summary types...")
    print("   (This may take 2-3 minutes for all summaries)")
    try:
        full_result = summarizer.process_log_content(
            complex_application_log,
            summary_types=['executive', 'technical', 'action_items']
        )
        if full_result['status'] == 'success':
            summaries = full_result['summaries']['summaries']
            if 'executive' in summaries:
                print("\n" + "=" * 60)
                print("🎯 EXECUTIVE SUMMARY (For Leadership)")
                print("=" * 60)
                print(summaries['executive'][:400] + "..." if len(summaries['executive']) > 400 else summaries['executive'])
            if 'action_items' in summaries:
                print("\n" + "=" * 60)
                print("📋 ACTION ITEMS (For Operations Teams)")
                print("=" * 60)
                print(summaries['action_items'][:400] + "..." if len(summaries['action_items']) > 400 else summaries['action_items'])
            if 'technical' in summaries:
                print("\n" + "=" * 60)
                print("🔧 TECHNICAL SUMMARY (For Engineering Teams)")
                print("=" * 60)
                print(summaries['technical'][:400] + "..." if len(summaries['technical']) > 400 else summaries['technical'])
            print("\n" + "=" * 80)
            print("✨ ALL SUMMARIES GENERATED SUCCESSFULLY!")
            print("   • Tailored for executive, technical, and operational needs")
            print("   • Actionable insights with clear ownership and timelines")
            print("   • Ready for immediate distribution to stakeholders")
        else:
            print(f"❌ Error generating summaries: {full_result['error']}")
    except Exception as e:
        print(f"❌ Unexpected error: {str(e)}")
        print("💡 This might be due to Ollama model not being available or network issues")
else:
    print("⚠️ Ollama not available - cannot generate multiple summary types")

2025-08-27 20:25:12,773 - __main__ - INFO - Processing log content...
2025-08-27 20:25:12,776 - __main__ - INFO - Generating executive summary...
2025-08-27 20:25:12,777 - __main__ - INFO - Calling Ollama API (attempt 1/3)
2025-08-27 20:25:12,776 - __main__ - INFO - Generating executive summary...
2025-08-27 20:25:12,777 - __main__ - INFO - Calling Ollama API (attempt 1/3)


⏳ Generating all summary types...
   (This may take 2-3 minutes for all summaries)


2025-08-27 20:27:02,036 - __main__ - INFO - Successfully generated summary (2124 characters)
2025-08-27 20:27:02,039 - __main__ - INFO - Generating technical summary...
2025-08-27 20:27:02,039 - __main__ - INFO - Calling Ollama API (attempt 1/3)
2025-08-27 20:27:02,039 - __main__ - INFO - Generating technical summary...
2025-08-27 20:27:02,039 - __main__ - INFO - Calling Ollama API (attempt 1/3)
2025-08-27 20:34:16,396 - __main__ - INFO - Successfully generated summary (3384 characters)
2025-08-27 20:34:16,459 - __main__ - INFO - Generating action_items summary...
2025-08-27 20:34:16,464 - __main__ - INFO - Calling Ollama API (attempt 1/3)
2025-08-27 20:34:16,396 - __main__ - INFO - Successfully generated summary (3384 characters)
2025-08-27 20:34:16,459 - __main__ - INFO - Generating action_items summary...
2025-08-27 20:34:16,464 - __main__ - INFO - Calling Ollama API (attempt 1/3)
2025-08-27 20:39:14,313 - __main__ - INFO - Successfully generated summary (2561 characters)
2025-08-27


🎯 EXECUTIVE SUMMARY (For Leadership)
**EXECUTIVE SUMMARY**

**Business Impact:**
The log analysis indicates a critical situation that may impact revenue, customer satisfaction, and operational efficiency. The failure to process payment transactions, data synchronization issues, security breaches, and application failures can lead to:

* Revenue loss due to delayed or failed transactions
* Customer dissatisfaction with slow or unavail...

📋 ACTION ITEMS (For Operations Teams)
**PRIORITIZED ACTION PLAN**

**IMMEDIATE ACTIONS (0-30 minutes)**

1. **Fix Critical Security Breach**
	* WHO: Security Team
	* WHAT: Block IP 192.168.1.100 and implement rate limiting for authentication attempts
	* WHEN: Immediately
	* RESOURCE REQUIREMENTS: None
	* SUCCESS METRICS: Monitor login attempts and block rates to ensure security breach is contained

2. **Resolve Fatal Error**
	* WHO: D...

🔧 TECHNICAL SUMMARY (For Engineering Teams)
**TECHNICAL SUMMARY**

**ROOT CAUSE ANALYSIS**

The root cause of the 